In [12]:
import scobra
import pandas as pd
import fluxtools.sbml_interface as si
import fluxtools.nlcm as nlcm
import suppFunctions as sf
import fluxtools.reaction_networks as rn
from fluxtools.functions import Linear
from fluxtools.nlcm import NonlinearNetworkModel
from fluxtools import simplification
from fluxtools.utilities import cobra

In [13]:
net = si.fromSBMLFile('conversion-test2.xml')
oldm1m2 = scobra.Model('AraLightDarkCore_NADPME_v4-3.xls')
reactions = net.reactions.copy()
reaction_ids = reactions.keys()
oldm = scobra.Model('AraLightDarkCore_ScrumPy.xls')

In [ ]:
#Setting biomasses to 0

lightdic = {'Alanine_Light_biomass':-9.7,'Arginine_Light_biomass':-1.6,'Asparagine_Light_biomass':-10.1,'Aspartate_Light_biomass':-9.5,
'Glutamate_Light_biomass':-11.4,'Glutamine_Light_biomass':-33.2,'Glycine_Light_biomass':-0.7,'Serine_Light_biomass':-7.7,
'Tyrosine_Light_biomass':-0.6,'Histidine_Light_biomass':-0.4,'Isoleucine_Light_biomass':-1.2,'Leucine_Light_biomass':-1.2,
'Lysine_Light_biomass':-1.8,'Methionine_Light_biomass':-0.7,'Phenylalanine_Light_biomass':-1.1,'Threonine_Light_biomass':-5.6,
'Tryptophan_Light_biomass':-0.9,'Valine_Light_biomass':-2.4}

darkdic = {'Alanine_Dark_biomass':-9.7,'Arginine_Dark_biomass':-1.6,'Asparagine_Dark_biomass':-10.1,'Aspartate_Dark_biomass':-9.5,
'Glutamate_Dark_biomass':-11.4,'Glutamine_Dark_biomass':-33.2,'Glycine_Dark_biomass':-0.7,'Serine_Dark_biomass':-7.7,
'Tyrosine_Dark_biomass':-0.6,'Histidine_Dark_biomass':-0.4,'Isoleucine_Dark_biomass':-1.2,'Leucine_Dark_biomass':-1.2,
'Lysine_Dark_biomass':-1.8,'Methionine_Dark_biomass':-0.7,'Phenylalanine_Dark_biomass':-1.1,'Threonine_Dark_biomass':-5.6,
'Tryptophan_Dark_biomass':-0.9,'Valine_Dark_biomass':-2.4}

exclude_list = []

for aa in lightdic.keys():
    M1 = aa+'_M1'
    exclude_list.append(M1)
    M2 = aa+'_M2'
    exclude_list.append(M2)

for aa in darkdic.keys():
    M1 = aa+'_M1'
    exclude_list.append(M1)
    M2 = aa+'_M2'
    exclude_list.append(M2)

exclude_list.append('Sucrose_Light_biomass_M1')
exclude_list.append('Sucrose_Light_biomass_M2')
exclude_list.append('Sucrose_Dark_biomass_M1')
exclude_list.append('Sucrose_Dark_biomass_M2')

list_b = oldm1m2.Reactions('_biomass')

# for i in list_b:
#     check = sf.notInList(i, exclude_list)
#     if check:
#         m1m2.SetConstraint(i,0,0)

In [ ]:
for i in range(len(exclude_list)):
    exclude_list[i] = 'R_'+exclude_list[i]


In [ ]:

for i in range(len(list_b)):
    list_b[i] = 'R_'+list_b[i]

In [ ]:
print list_b

In [ ]:
m1m2 = NonlinearNetworkModel('pleasework', net)

constraints = ['Photon_Dark_tx',
                   'GLC_Dark_tx',
                   'GLC_Light_tx',
                   'Sucrose_Dark_tx',
                   'Sucrose_Light_tx',
                   'TAG_Dark_tx',
                   'TAG_Light_tx',
                   'Starch_Dark_tx',
                   'Starch_Light_tx',
                   'NADPH_Dehydrogenase_Dark_p',
                   'NADPH_Dehydrogenase_Light_p',
                   'NADPHoxx_Light_tx',
                    'NADPHoxx_Dark_tx',
                    'Plastoquinol_Oxidase_Dark_p',
                    'Plastoquinol_Oxidase_Light_p',
                    'NH4_Light_tx',
                    'NH4_Dark_tx']

for i in range(len(constraints)):
    constraints[i] = 'R_'+constraints[i]
    
print constraints

for tx in constraints:
    reac = tx+'_M1'
    m1m2.set_bound(reac,(0,0))

for tx in constraints:
    reac = tx+'_M2'
    print reac
    m1m2.set_bound(reac,(0,0))

for i in list_b:
    check = sf.notInList(i, exclude_list)
    if check:
        m1m2.set_bound(i,(0,0))

    
m1m2.add_constraint('carbox/oxy1', "R_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_Light_p_M1/R_RXN_961_Light_p_M1",3)
m1m2.add_constraint('carbox/oxy2', "R_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_Dark_p_M1/R_RXN_961_Dark_p_M1",3)
m1m2.add_constraint('NO3M1','R_NO3_Light_tx_M1/R_NO3_Dark_tx_M1',3/2)
m1m2.add_constraint('NO3M2','R_NO3_Light_tx_M2/R_NO3_Dark_tx_M2',3/2)


#Set ATP and NADPH rxn to be None to 0. 
ATP_constraints = ['R_NADPHoxc', 'R_NADPHoxm', 'R_NADPHoxp']

m1m2.add_constraint('ATP_NADPh1','R_NADPHoxc_Light_tx_M1/R_ATPase_Light_tx_M1',1/9)
m1m2.add_constraint('ATP_NADPh2','R_NADPHoxc_Light_tx_M2/R_ATPase_Light_tx_M2',1/9)
m1m2.add_constraint('ATP_NADPh3','R_NADPHoxm_Light_tx_M1/R_ATPase_Light_tx_M1',1/9)
m1m2.add_constraint('ATP_NADPh4','R_NADPHoxm_Light_tx_M2/R_ATPase_Light_tx_M2',1/9)
m1m2.add_constraint('ATP_NADPh5','R_NADPHoxp_Light_tx_M1/R_ATPase_Light_tx_M1',1/9)
m1m2.add_constraint('ATP_NADPh6','R_NADPHoxp_Light_tx_M1/R_ATPase_Light_tx_M1',1/9)
m1m2.add_constraint('ATP_NADPh7','R_NADPHoxc_Dark_tx_M1/R_ATPase_Light_tx_M1',1/9)
m1m2.add_constraint('ATP_NADPh8','R_NADPHoxc_Dark_tx_M2/R_ATPase_Light_tx_M2',1/9)
m1m2.add_constraint('ATP_NADPh9','R_NADPHoxp_Dark_tx_M1/R_ATPase_Light_tx_M1',1/9)
m1m2.add_constraint('ATP_NADPh10','R_NADPHoxp_Dark_tx_M2/R_ATPase_Light_tx_M2',1/9)
m1m2.add_constraint('ATP_NADPh11','R_NADPHoxm_Dark_tx_M1/R_ATPase_Light_tx_M1',1/9)
m1m2.add_constraint('ATP_NADPh12','R_NADPHoxm_Dark_tx_M2/R_ATPase_Light_tx_M2',1/9)



# for reac in ATP_constraints:
#     nadph_d = reac+'_Light_tx_M1'
#     nadph_n = reac+'_Dark_tx_M1'
    
#     m1m2.add_constraint('ATP_NADPh',nadph_d+'/R_ATPase_Light_tx_M1',1/9)
#     m1m2.add_constraint('ATP_NADPh',nadph_n+'/R_ATPase_Dark_tx_M1',1/9)

    
# for reac in ATP_constraints:
#     nadph_d = reac+'_Light_tx_M2'
#     nadph_n = reac+'_Dark_tx_M2'
    
#     m1m2.add_constraint('ATP_NADPh',nadph_d+'/R_ATPase_Light_tx_M2',1/9)
#     m1m2.add_constraint('ATP_NADPh',nadph_n+'/R_ATPase_Dark_tx_M2',1/9)
    

# m1m2.add_constraint('sucrose1','R_Sucrose_Light_biomass_M1/R_Sucrose_Dark_biomass_M1',3)
# m1m2.add_constraint('sucrose2','R_Sucrose_Light_biomass_M2/R_Sucrose_Dark_biomass_M2',3)  


# # Setting ratios between sucrose and amino acids
# for aa in lightdic.keys():
#     m1m2.add_constraint('aa_sucrose','R_'+aa+'_M1/R_Sucrose_Light_biomass_M1', -lightdic[aa]/220)
#     m1m2.add_constraint('aa_sucrose','R_'+aa+'_M2/R_Sucrose_Light_biomass_M2', -lightdic[aa]/220)

# for aa in darkdic.keys():
#     m1m2.add_constraint('aa_sucrose','R_'+aa+'_M1/R_Sucrose_Dark_biomass_M1', -darkdic[aa]/220)
#     m1m2.add_constraint('aa_sucrose','R_'+aa+'_M2/R_Sucrose_Dark_biomass_M2', -darkdic[aa]/220)


m1m2.set_bound('R_ATPase_Light_tx_M1',(-7.5,-7.5))
m1m2.set_bound('R_ATPase_Dark_tx_M1',(-7.5,-7.5))
m1m2.set_bound('R_ATPase_Light_tx_M2',(-7.5,-7.5))
m1m2.set_bound('R_ATPase_Dark_tx_M2',(-7.5,-7.5))



m1m2.set_objective('max_Sucrose_export','-1*R_Sucrose_Light_biomass_M2')
m1m2.set_objective('max_Sucrose_export2','-1*R_Sucrose_Dark_biomass_M2')
m1m2.set_bound('R_Photon_Light_tx_M1',(0,200))
m1m2.set_bound('R_Photon_Light_tx_M2',(0,200))
model.set_bound('R_Sucrose_Light_biomass_M1',(0,400))
model.set_bound('R_Sucrose_Light_biomass_M2',(0,400))


m1m2.compile()
print 'done'
m1m2.solve()
for i in reaction_ids:
    print i, m1m2.soln[i]



In [17]:
from fluxtools.utilities import cobra

m = si.fromSBMLFile('m_conversion_test3.xml')
reactions = m.reactions.copy()
reaction_ids = reactions.keys()
# reaction_ids
model = nlcm.NonlinearNetworkModel('bla', m)
# nlcm.default_ipopt_options = {
#     'print_level': 5,
#     'tol': 1e-5,
#     'linear_solver': 'ma97',
#     'ma97_print_level': -1,
#     'max_iter': 500
# }

In [18]:
boundaries = cobra.get_flux_bounds(m)

# print boundaries['R_ATPase_Light_tx']
for i in boundaries:
#     print i
    model.set_bound(i,boundaries[i])

In [23]:
constraints = ['Photon_Dark_tx',
                   'GLC_Dark_tx',
                   'GLC_Light_tx',
                   'Sucrose_Dark_tx',
                   'Sucrose_Light_tx',
                   'TAG_Dark_tx',
                   'TAG_Light_tx',
                   'Starch_Dark_tx',
                   'Starch_Light_tx',
                   'NADPH_Dehydrogenase_Dark_p',
                   'NADPH_Dehydrogenase_Light_p',
                   'NADPHoxx_Light_tx',
                    'NADPHoxx_Dark_tx',
                    'Plastoquinol_Oxidase_Dark_p',
                    'Plastoquinol_Oxidase_Light_p',
                    'NH4_Light_tx',
                    'NH4_Dark_tx']

for tx in constraints:
    reac = 'R_'+tx
    model.set_bound(reac,(0,0))


#Setting selected biomasses to 0
exclude_list = []
list_b = []
lightdic = {'Alanine_Light_biomass':-9.7,'Arginine_Light_biomass':-1.6,'Asparagine_Light_biomass':-10.1,'Aspartate_Light_biomass':-9.5,
'Glutamate_Light_biomass':-11.4,'Glutamine_Light_biomass':-33.2,'Glycine_Light_biomass':-0.7,'Serine_Light_biomass':-7.7,
'Tyrosine_Light_biomass':-0.6,'Histidine_Light_biomass':-0.4,'Isoleucine_Light_biomass':-1.2,'Leucine_Light_biomass':-1.2,
'Lysine_Light_biomass':-1.8,'Methionine_Light_biomass':-0.7,'Phenylalanine_Light_biomass':-1.1,'Threonine_Light_biomass':-5.6,
'Tryptophan_Light_biomass':-0.9,'Valine_Light_biomass':-2.4}

darkdic = {'Alanine_Dark_biomass':-9.7,'Arginine_Dark_biomass':-1.6,'Asparagine_Dark_biomass':-10.1,'Aspartate_Dark_biomass':-9.5,
'Glutamate_Dark_biomass':-11.4,'Glutamine_Dark_biomass':-33.2,'Glycine_Dark_biomass':-0.7,'Serine_Dark_biomass':-7.7,
'Tyrosine_Dark_biomass':-0.6,'Histidine_Dark_biomass':-0.4,'Isoleucine_Dark_biomass':-1.2,'Leucine_Dark_biomass':-1.2,
'Lysine_Dark_biomass':-1.8,'Methionine_Dark_biomass':-0.7,'Phenylalanine_Dark_biomass':-1.1,'Threonine_Dark_biomass':-5.6,
'Tryptophan_Dark_biomass':-0.9,'Valine_Dark_biomass':-2.4}


for i in lightdic.keys():
    exclude_list.append('R_'+i)

for i in darkdic.keys():
    exclude_list.append('R_'+i)

exclude_list.append('R_Sucrose_Light_biomass')
exclude_list.append('R_Sucrose_Dark_biomass')

list_b = oldm.Reactions('_biomass')


for i in range(len(exclude_list)):
    exclude_list[i] = exclude_list[i]

for i in range(len(list_b)):
    list_b[i] = list_b[i]

# print exclude_list

for i in list_b:
    check = sf.notInList(i, exclude_list)
    if check:
        model.set_bound(i,(0,0))

# #Adding NO3 Constraints




# # #Set ATP and NADPH rxn to be None to 0. 
# # ATP_constraints = ['R_NADPHoxc', 'R_NADPHoxm', 'R_NADPHoxp']

model.set_bound('R_ATPase_Light_tx',(-7.5,-7.5))
model.set_bound('R_ATPase_Dark_tx',(-7.5,-7.5))

# model.add_constraint('ATP_NADPh1','R_ATPase_Light_tx/R_NADPHoxc_Light_tx',9)
# model.add_constraint('ATP_NADPh3','R_ATPase_Light_tx/R_NADPHoxm_Light_tx',9)
# model.add_constraint('ATP_NADPh6','R_ATPase_Light_tx/R_NADPHoxp_Light_tx',9)
# model.add_constraint('ATP_NADPh7','R_ATPase_Dark_tx/R_NADPHoxc_Dark_tx',9)
# model.add_constraint('ATP_NADPh9','R_ATPase_Dark_tx/R_NADPHoxm_Dark_tx',9)
# model.add_constraint('ATP_NADPh11','R_ATPase_Dark_tx/R_NADPHoxp_Dark_tx',9)

# model.add_constraint('ATP_NADPh1','R_NADPHoxc_Light_tx/R_ATPase_Light_tx',1/9)
# model.add_constraint('ATP_NADPh3','R_NADPHoxm_Light_tx/R_ATPase_Light_tx',1/9)
# model.add_constraint('ATP_NADPh6','R_NADPHoxp_Light_tx/R_ATPase_Light_tx',1/9)
# model.add_constraint('ATP_NADPh7','R_NADPHoxc_Dark_tx/R_ATPase_Dark_tx',1/9)
# model.add_constraint('ATP_NADPh9','R_ATPase_Dark_tx/R_NADPHoxp_Dark_tx',1/9)
# model.add_constraint('ATP_NADPh11','R_ATPase_Dark_tx/R_NADPHoxm_Dark_tx',1/9)

i = 0
for aa in lightdic.keys():
    x = 220/-lightdic[aa]
    model.add_constraint('aa_constraint'+str(i), 'R_Sucrose_Light_biomass/R_'+aa, x)
    i+=1

i = 0
for aa in darkdic.keys():
#     print 'R_'+aa
    x = 220/-darkdic[aa]
    model.add_constraint('aa_constraint'+str(i), 'R_Sucrose_Dark_biomass/R_'+aa, x)
    i+=1

model.add_constraint('sucrose1','R_Sucrose_Light_biomass/R_Sucrose_Dark_biomass',3)
model.add_constraint('NO3','R_NO3_Light_tx/R_NO3_Dark_tx',1.5)
# model.remove_constraint('sucrose1')
model.set_objective('max_sucrose_exportL','-1.0*R_Sucrose_Light_biomass')
model.add_constraint('Carbox_oxyge', 'R_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_Light_p/R_RXN_961_Light_p',3.)
model.set_bound('R_Photon_Light_tx',(0,1000))
model.set_bound('R_Photon_Dark_tx',(0,0))
model.set_bound('R_Sucrose_Light_biomass',(-0.5,1000))
model.set_bound('R_PSII_RXN_Light_p',(0,1000))
# model.set_bound('R_Mitochondrial_ATP_Synthase_Light_m',(0,0))


    
model.compile()
model.solve()
print 'photon:', model.soln['R_Photon_Light_tx']
print 'PSII:', model.soln['R_PSII_RXN_Light_p']
print 'CO2_tx:', model.soln['R_CO2_Dark_tx']
print 'O2_tx:', model.soln['R_O2_Light_tx']
print 'sucrose Light out:',model.soln['R_Sucrose_Light_biomass']
print 'sucrose Dark out:',model.soln['R_Sucrose_Dark_biomass']
print 'Alanine Light out:',model.soln['R_Alanine_Light_biomass']
print 'Alanine Dark out:', model.soln['R_Alanine_Dark_biomass']
print 'carbox:',model.soln['R_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_Light_p']
print 'oxyg:',model.soln['R_RXN_961_Light_p']
print 'NO3_light:',model.soln['R_NO3_Light_tx']
print 'NO3_dark:',model.soln['R_NO3_Dark_tx']
print 'GPH-rxn:',model.soln['R_GPH_RXN_Light_p']
print 'ATPase:', model.soln['R_ATPase_Light_tx']
print 'GLC:', model.soln['R_GLC_Light_tx']
print 'PEP:', model.soln['R_PEPCARBOX_RXN_Light_c']
# for i in reaction_ids:
#     print i, model.soln[i]



print '\n'
model.soln


OptimizationFailure: IPOPT exited with status -1

In [20]:
model.get_bounds('R_Sucrose_Light_tx')

(0, 0)

In [ ]:
#IT WORKS!
model.set_objective('sucrose_export','1.0*R_Sucrose_Light_biomass')
model.add_constraint('CO2_leakiness_constraint', 'R_CO2_Light_tx/R_O2_Light_tx',1.)
model.add_constraint('CO2_leakiness_constraint', 'R_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_Light_p/R_RXN_961_Light_p',3)
model.set_bound('R_Photon_Light',(0,200))
model.set_bound('R_Sucrose_Light_biomass',(-10,0))
model.set_bound('R_Photon_Dark_tx',(0,0))


model.compile()
model.solve()
model.soln
# for i in reaction_ids:
#     print i, model.soln[i]

In [ ]:
simple_model, details = simplification.simplify(model)
simple_model.compile()
simple_model.solve()
# print simple_model.soln['R_Photon_Light_tx']
print simple_model.soln['R_Sucrose_Light_biomass']
print simple_model.soln['R_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_Light_p']
print simple_model.soln['R_RXN_961_Light_p']


In [ ]:
details


In [ ]:
simple_model.soln['R_Photon_Light_tx']